In [0]:
dbutils.fs.mount(
  source = "wasbs://retail@tjnewstorage71.blob.core.windows.net",
  mount_point = "/mnt/retail_project",
  extra_configs = {"fs.azure.account.key.tjnewstorage71.blob.core.windows.net":"0jySIqf08hlhsGUQnWfgV5K5YJox6t3WjhkeKp385VRjNi7m4NlUvnm9PrMazTRQGlZ+SKP/zDj1+AStXgmPXQ=="})

True

In [0]:
dbutils.fs.ls('/mnt/retail_project/')

[FileInfo(path='dbfs:/mnt/retail_project/bronze/', name='bronze/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/retail_project/gold/', name='gold/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/retail_project/silver/', name='silver/', size=0, modificationTime=0)]

In [0]:
df_transaction = spark.read.parquet('/mnt/retail_project/bronze/transaction/')
df_products = spark.read.parquet('/mnt/retail_project/bronze/product/')
df_stores = spark.read.parquet('/mnt/retail_project/bronze/store/')
df_customers = spark.read.parquet('/mnt/retail_project/bronze/customer/manish040596/azure-data-engineer---multi-source/refs/heads/main/')

display(df_products)



product_id,product_name,category,price
1,Wireless Mouse,Electronics,799.99
2,Bluetooth Speaker,Electronics,1299.49
3,Yoga Mat,Fitness,499.00
4,Laptop Stand,Accessories,999.99
5,Notebook Set,Stationery,149.00
6,Water Bottle,Fitness,299.00
7,Smartwatch,Electronics,4999.00
8,Desk Organizer,Accessories,399.00
9,Dumbbell Set,Fitness,1999.00
10,Pen Drive 32GB,Electronics,599.00


In [0]:
# Converting Data Types

from pyspark.sql.functions import col

df_transaction = df_transaction.select(
    col("transaction_id").cast("int"),
    col("customer_id").cast("int"),
    col("product_id").cast("int"),
    col("store_id").cast("int"),
    col("quantity").cast("int"),
    col("transaction_date").cast("date")
)

df_products = df_products.select(
    col("product_id").cast("int"),
    col("product_name"),
    col("category"),
    col("price").cast("double")
)

df_stores = df_stores.select(
    col("store_id").cast("int"),
    col("store_name"),
    col("location")
)

df_customers = df_customers[0].select(
    "customer_id","first_name","last_name","email","phone","city","registration_date"
).dropDuplicates(["customer_id"])

In [0]:
df_silver = (
    df_transaction
    .join(df_customers, "customer_id")
    .join(df_products, "product_id")
    .join(df_stores, "store_id")
    .withColumn("total_amount", col("quantity") * col("price"))
)

In [0]:
display(df_silver)

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,phone,city,registration_date,product_name,category,price,store_name,location,total_amount
1,5,130,20,2,2024-07-30,Tanvi,Kapoor,user130@example.com,9598765432,Pune,2023-12-22,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,298.0
2,6,126,26,2,2024-09-21,Bhavna,Nair,user126@example.com,9632109876,Mumbai,2023-07-17,Water Bottle,Fitness,299.0,High Street Store,Delhi,598.0
2,7,110,7,5,2025-01-01,Deepak,Nair,user110@example.com,9798765432,Mumbai,2023-10-14,Smartwatch,Electronics,4999.0,High Street Store,Delhi,24995.0
4,9,119,29,2,2025-06-03,Kunal,Nair,user119@example.com,9709876543,Bangalore,2023-08-10,Dumbbell Set,Fitness,1999.0,Downtown Mini Store,Pune,3998.0
4,8,116,30,5,2025-03-16,Rakesh,Kapoor,user116@example.com,9732109876,Kolkata,2023-06-15,Desk Organizer,Accessories,399.0,Downtown Mini Store,Pune,1995.0
2,2,124,10,5,2024-08-27,Kavita,Sharma,user124@example.com,9654321098,Kolkata,2023-11-15,Bluetooth Speaker,Electronics,1299.49,High Street Store,Delhi,6497.45
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,9821098765,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.0,Tech World Outlet,Bangalore,7996.0
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,9865432109,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.9700000000003
3,3,104,13,4,2025-05-04,Karan,Patel,user104@example.com,9854321098,Hyderabad,2024-02-05,Yoga Mat,Fitness,499.0,Tech World Outlet,Bangalore,1996.0
4,1,120,14,5,2024-07-17,Alka,Mishra,user120@example.com,9698765432,Hyderabad,2023-12-01,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,3999.95


In [0]:
silver_path = "/mnt/retail_project/silver/"
df_silver.write.mode("overwrite").format("delta").save(silver_path)

In [0]:
spark.sql(f"""
CREATE TABLE retail_silver_cleaned
USING DELTA
LOCATION '/mnt/retail_project/silver/'
""")

DataFrame[]

In [0]:
%sql select * from retail_silver_cleaned

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,first_name,last_name,email,phone,city,registration_date,product_name,category,price,store_name,location,total_amount
1,5,130,20,2,2024-07-30,Tanvi,Kapoor,user130@example.com,9598765432,Pune,2023-12-22,Notebook Set,Stationery,149.0,City Mall Store,Mumbai,298.0
2,6,126,26,2,2024-09-21,Bhavna,Nair,user126@example.com,9632109876,Mumbai,2023-07-17,Water Bottle,Fitness,299.0,High Street Store,Delhi,598.0
2,7,110,7,5,2025-01-01,Deepak,Nair,user110@example.com,9798765432,Mumbai,2023-10-14,Smartwatch,Electronics,4999.0,High Street Store,Delhi,24995.0
4,9,119,29,2,2025-06-03,Kunal,Nair,user119@example.com,9709876543,Bangalore,2023-08-10,Dumbbell Set,Fitness,1999.0,Downtown Mini Store,Pune,3998.0
4,8,116,30,5,2025-03-16,Rakesh,Kapoor,user116@example.com,9732109876,Kolkata,2023-06-15,Desk Organizer,Accessories,399.0,Downtown Mini Store,Pune,1995.0
2,2,124,10,5,2024-08-27,Kavita,Sharma,user124@example.com,9654321098,Kolkata,2023-11-15,Bluetooth Speaker,Electronics,1299.49,High Street Store,Delhi,6497.45
3,9,107,22,4,2024-11-16,Priya,Kapoor,user107@example.com,9821098765,Ahmedabad,2023-05-12,Dumbbell Set,Fitness,1999.0,Tech World Outlet,Bangalore,7996.0
4,1,103,18,3,2024-09-05,Sonal,Sharma,user103@example.com,9865432109,Bangalore,2023-07-10,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,2399.9700000000003
3,3,104,13,4,2025-05-04,Karan,Patel,user104@example.com,9854321098,Hyderabad,2024-02-05,Yoga Mat,Fitness,499.0,Tech World Outlet,Bangalore,1996.0
4,1,120,14,5,2024-07-17,Alka,Mishra,user120@example.com,9698765432,Hyderabad,2023-12-01,Wireless Mouse,Electronics,799.99,Downtown Mini Store,Pune,3999.95


In [0]:
silver_df = spark.read.format("delta").load("/mnt/retail_project/silver/")


In [0]:
from pyspark.sql.functions import sum, countDistinct, avg

gold_df = silver_df.groupBy(
    "transaction_date",
    "product_id", "product_name", "category",
    "store_id", "store_name", "location"
).agg(
    sum("quantity").alias("total_quantity_sold"),
    sum("total_amount").alias("total_sales_amount"),
    countDistinct("transaction_id").alias("number_of_transactions"),
    avg("total_amount").alias("average_transaction_value")
)

In [0]:
display(gold_df)

transaction_date,product_id,product_name,category,store_id,store_name,location,total_quantity_sold,total_sales_amount,number_of_transactions,average_transaction_value
2024-11-02,8,Desk Organizer,Accessories,1,City Mall Store,Mumbai,1,399.0,1,399.0
2024-08-11,1,Wireless Mouse,Electronics,3,Tech World Outlet,Bangalore,2,1599.98,1,1599.98
2024-12-13,8,Desk Organizer,Accessories,4,Downtown Mini Store,Pune,5,1995.0,1,1995.0
2025-05-04,3,Yoga Mat,Fitness,3,Tech World Outlet,Bangalore,4,1996.0,1,1996.0
2025-05-26,5,Notebook Set,Stationery,1,City Mall Store,Mumbai,4,596.0,1,596.0
2024-07-14,1,Wireless Mouse,Electronics,5,Mega Plaza,Chennai,1,799.99,1,799.99
2024-07-17,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,5,3999.95,1,3999.95
2024-09-05,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,3,2399.9700000000003,1,2399.9700000000003
2025-06-03,9,Dumbbell Set,Fitness,4,Downtown Mini Store,Pune,2,3998.0,1,3998.0
2024-08-27,2,Bluetooth Speaker,Electronics,2,High Street Store,Delhi,5,6497.45,1,6497.45


In [0]:
gold_path = "/mnt/retail_project/gold/"

gold_df.write.mode("overwrite").format("delta").save(gold_path)

In [0]:
spark.sql("""
CREATE TABLE retail_gold_sales_summary
USING DELTA
LOCATION '/mnt/retail_project/gold/' """)

DataFrame[]

In [0]:
%sql select * from retail_gold_sales_summary

transaction_date,product_id,product_name,category,store_id,store_name,location,total_quantity_sold,total_sales_amount,number_of_transactions,average_transaction_value
2024-11-02,8,Desk Organizer,Accessories,1,City Mall Store,Mumbai,1,399.0,1,399.0
2024-08-11,1,Wireless Mouse,Electronics,3,Tech World Outlet,Bangalore,2,1599.98,1,1599.98
2024-12-13,8,Desk Organizer,Accessories,4,Downtown Mini Store,Pune,5,1995.0,1,1995.0
2025-05-04,3,Yoga Mat,Fitness,3,Tech World Outlet,Bangalore,4,1996.0,1,1996.0
2025-05-26,5,Notebook Set,Stationery,1,City Mall Store,Mumbai,4,596.0,1,596.0
2024-07-14,1,Wireless Mouse,Electronics,5,Mega Plaza,Chennai,1,799.99,1,799.99
2024-07-17,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,5,3999.95,1,3999.95
2024-09-05,1,Wireless Mouse,Electronics,4,Downtown Mini Store,Pune,3,2399.9700000000003,1,2399.9700000000003
2025-06-03,9,Dumbbell Set,Fitness,4,Downtown Mini Store,Pune,2,3998.0,1,3998.0
2024-08-27,2,Bluetooth Speaker,Electronics,2,High Street Store,Delhi,5,6497.45,1,6497.45
